In [2]:
import k3d
import os
import vtk
import numpy as np
from k3d.helpers import download
from vtk.util import numpy_support
import pyvista as pv
import math


def vtk_ExtractSurface(vtk_outputport,vtk_o,vtk_n):

    plane = vtk.vtkPlane()
    plane.SetOrigin(*vtk_o)
    plane.SetNormal(*vtk_n)

    myExtractGeometry = vtk.vtkExtractGeometry()
    myExtractGeometry.SetInputData(vtk_outputport)
    myExtractGeometry.SetImplicitFunction(plane)
    myExtractGeometry.ExtractInsideOn()
    myExtractGeometry.SetExtractBoundaryCells(0);
    myExtractGeometry.Update()

    myExtractSurface = vtk.vtkDataSetSurfaceFilter()
    myExtractSurface.SetInputConnection(myExtractGeometry.GetOutputPort())
    myExtractSurface.Update()

    return myExtractSurface.GetOutput()


def read_unstr (name):
    filename = name
    reader = vtk.vtkXMLMultiBlockDataReader()
    reader.SetFileName(filename)
    reader.Update()
    #out = reader.GetOutput()
    #outunst = pv.UnstructuredGrid(out.GetBlock(0).GetBlock(0)) 
    #bbox = np.array(outunst.GetBounds()).reshape(3,2)
    #center = np.mean(bbox,axis=1)
    #plane = vtk.vtkPlane()
    #plane.SetOrigin(*center)
    #plane.SetNormal(1,0.3,0)
    #vtk_n = np.array([ 0. ,  .3,  0. ])
    #vtk_o = np.array([ 0.04984861,  20.03934663,  0.04888905])    
   #return vtk_ExtractSurface(outunst, vtk_o,vtk_n)
    #return outunst.extract_geometry()



# read the data
grid = pv.read('surf0.stl')
grid2 = pv.read('surf10.stl')


c = read_unstr ('block0.vtm')
c2 = read_unstr ('block1.vtm')



colors = [0xFF0000,0xaaaaaa,0xFF0000]
grids = [grid,grid,grid2]
conturs = [c,c,c2]
cl_at = ('T', 0.0, 2000)
cl_at1 = ('T', 0.0, 2000)
cl_at2 = ('T', 0.0, 2000)

ra = [cl_at1[1:3],cl_at[1:3],cl_at2[1:3]]


plot = k3d.plot()

plt_vtk = k3d.vtk_poly_data(c ,color_attribute = cl_at, color_map = k3d.basic_color_maps.Jet)
plt_mesh = k3d.vtk_poly_data(grid2, opacity=0.05, wireframe=True,color = 0xaaaaaa)

plot += plt_mesh
plot += plt_vtk


#c_color_range = {str(t):ra[t] for t in range(len(conturs))}
attribute = {str(t):numpy_support.vtk_to_numpy(conturs[t].GetPointData().GetArray(cl_at[0])) for t in range(len(conturs))}
vertices = {str(t):numpy_support.vtk_to_numpy(grids[t].GetPoints().GetData()).astype(np.float32) for t in range(len(grids))}
indices = {str(t):numpy_support.vtk_to_numpy(grids[t].GetPolys().GetData()).reshape(-1, 4)[:, 1:4].astype(np.float32) for t in range(len(grids))}

#print (attribute)


plt_mesh.indices,plt_mesh.vertices = indices,vertices

plt_vtk.attribute = attribute 
plot.fps = 1

plot.display()


Output()

In [6]:
f = open('./volume.html', 'w', encoding='UTF-8')
f.write(plot.get_snapshot(9, 'K3DInstance.startAutoPlay();'))
f.close()